# Tensorboard

In [1]:
import tensorflow as tf
import numpy as np

x = np.linspace(0, 2*np.pi, 100)
y = np.cos(x)

summary_writer = tf.summary.create_file_writer('tensorboard_log')

with summary_writer.as_default():
    for i, j in enumerate(y):
        tf.summary.scalar('cos', j, step=i)

In [3]:
import tensorflow as tf
import numpy as np

summary_writer = tf.summary.create_file_writer('tensorboard_log2')
data = tf.random.normal([64, 100], dtype=tf.float64)
with summary_writer.as_default():
    for i, offset in enumerate(tf.range(0, 10, delta=0.1, dtype=tf.float64)):
        tf.summary.histogram('Normal distribution', data+offset, step=i)

In [2]:
import tensorflow as tf
import datetime

mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0


def create_model():
    return tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation='softmax')
    ])


model = create_model()

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

train_dataset = train_dataset.shuffle(60000).batch(64)
test_dataset = test_dataset.batch(64)

loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# keras.metrics 衡量模型
train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')
test_loss = tf.keras.metrics.Mean('test_loss', dtype=tf.float32)
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')


def train_step(x_train, y_train):
    with tf.GradientTape() as tape:
        predictions = model(x_train, training=True)
        loss = loss_object(y_train, predictions)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    train_loss(loss)
    train_accuracy(y_train, predictions)


def test_step(x_test, y_test):
    predictions = model(x_test)
    loss = loss_object(y_test, predictions)

    test_loss(loss)
    test_accuracy(y_test, predictions)


current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

EPOCHS = 5

for epoch in range(EPOCHS):
    for (x_train, y_train) in train_dataset:
        train_step(x_train, y_train)
    with train_summary_writer.as_default():
        tf.summary.scalar('loss', train_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

    for (x_test, y_test) in test_dataset:
        test_step(x_test, y_test)
    with test_summary_writer.as_default():
        tf.summary.scalar('loss', test_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100,
                          test_loss.result(),
                          test_accuracy.result() * 100))

    # Reset metrics every epoch
    train_loss.reset_states()
    test_loss.reset_states()
    train_accuracy.reset_states()
    test_accuracy.reset_states()

Epoch 1, Loss: 0.2440073937177658, Accuracy: 92.9050064086914, Test Loss: 0.12156160920858383, Test Accuracy: 96.30000305175781
Epoch 2, Loss: 0.10694970190525055, Accuracy: 96.78166198730469, Test Loss: 0.08532525599002838, Test Accuracy: 97.25
Epoch 3, Loss: 0.07145542651414871, Accuracy: 97.80332946777344, Test Loss: 0.06910397112369537, Test Accuracy: 97.81999969482422
Epoch 4, Loss: 0.05492877587676048, Accuracy: 98.29666900634766, Test Loss: 0.06346160918474197, Test Accuracy: 98.02999877929688
Epoch 5, Loss: 0.04310205206274986, Accuracy: 98.63500213623047, Test Loss: 0.06564893573522568, Test Accuracy: 97.98999786376953


# Tensorboard (keras)

In [1]:
import tensorflow as tf

mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0


model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])


model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="fit_logs/", histogram_freq=1)

model.fit(x=x_train,
          y=y_train,
          epochs=5,
          validation_data=(x_test, y_test),
          callbacks=[tensorboard_callback])

Epoch 1/5
   1/1875 [..............................] - ETA: 0s - loss: 2.4814 - accuracy: 0.0938WARNING:tensorflow:From C:\Users\user\anaconda3\envs\tf2\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2187 - accuracy: 0.9345 - val_loss: 0.1155 - val_accuracy: 0.9653
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0935 - accuracy: 0.9715 - val_loss: 0.0838 - val_accuracy: 0.9723
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0697 - accuracy: 0.9782 - val_loss: 0.0708 - val_accuracy: 0.9787
Epoch 4/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0530 - accuracy: 0.9827 - val_loss: 0.0643 - val_accuracy: 0.9816
Epoch 5/5
1875/1875 [==============================]